# Import

In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px

pd.set_option('display.max_columns', None)

# Data Load

In [2]:
train = pd.read_csv('../new_open/train.csv')
test = pd.read_csv('../new_open/test.csv')

In [3]:
train.head()

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE,CI_HOUR
0,TRAIN_000000,SG,GIW5,Container,30.881018,2018-12-17 21:29,Z618338,30.0,24,24300,10.0,10.0,16700,180.0,CQSB78,Panama,NaN,NaN,NaN,NaN,5,0.002615,3.450000
1,TRAIN_000001,IN,UJM2,Bulk,0.000000,2014-09-23 6:59,X886125,30.0,13,35900,10.0,10.0,23500,180.0,SPNO34,Marshall Islands,NaN,NaN,NaN,NaN,12,0.000217,0.000000
2,TRAIN_000002,CN,EUC8,Container,0.000000,2015-02-03 22:00,T674582,50.0,12,146000,30.0,20.0,140000,370.0,FNPK22,Malta,NaN,NaN,NaN,NaN,6,0.001614,0.000000
3,TRAIN_000003,JP,ZAG4,Container,0.000000,2020-01-17 4:02,Y847238,20.0,18,6910,10.0,10.0,5400,120.0,PBZV77,Bahamas,-3.18,-1.61,6.7,2.629350,13,0.000356,0.000000
4,TRAIN_000004,SG,GIW5,Container,27.037650,2020-01-26 7:51,A872328,50.0,10,116000,20.0,10.0,96600,300.0,GUCE76,Liberia,-0.33,-3.28,25.6,2.495953,15,0.002615,253.554444


In [4]:
test.head()

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE
0,TEST_000000,SG,GIW5,Container,1.826589,2020-06-18 11:58,K322654,50.0,18,117000,30.0,20.0,109000,340.0,KQMD74,Panama,0.37,1.63,27.1,1.587063,19,0.002615
1,TEST_000001,CN,WEY7,Cargo,25.399386,2021-05-26 22:20,E376681,10.0,13,3810,10.0,10.0,2560,80.0,LBYS27,Belize,-2.79,-2.33,14.2,2.663972,6,0.001028
2,TEST_000002,CN,NGG6,Container,111.079467,2019-12-16 0:09,F811111,20.0,26,10900,10.0,10.0,8440,140.0,EKFV15,Singapore,0.04,-4.91,9.3,3.255315,8,0.001743
3,TEST_000003,CA,FFM2,Bulk,9.175258,2015-11-16 5:30,A737561,30.0,9,55800,20.0,10.0,31500,190.0,MASW32,Panama,NaN,NaN,NaN,NaN,2,0.000182
4,TEST_000004,JP,QYY1,Container,0.000000,2018-10-24 1:11,A827175,30.0,19,39800,20.0,10.0,33000,220.0,SIEL54,Liberia,NaN,NaN,NaN,NaN,10,0.000552


# 알고리즘

## train

In [5]:
train = train[['SAMPLE_ID', 'ARI_CO', 'ARI_PO', 'ATA', 'U_WIND', 'V_WIND', 'BN', 'AIR_TEMPERATURE', 'CI_HOUR']]

In [6]:
train['ATA'] = pd.to_datetime( train['ATA'])
train['CI_HOUR'] = pd.to_timedelta(train['CI_HOUR'], unit="hours")

In [7]:
# train에서 ARI_CO, ARI_PO, ATA가 서로 같은 데이터를 찾아서 dup_train에 저장
dup_train = train[(train.duplicated(['ARI_CO', 'ARI_PO', 'ATA'], keep=False)) & (train['ATA'] > '2020-01-01')].sort_values(by=['ARI_CO', 'ARI_PO', 'ATA'])
dup_train

,SAMPLE_ID,ARI_CO,ARI_PO,ATA,U_WIND,V_WIND,BN,AIR_TEMPERATURE,CI_HOUR
4827,TRAIN_004827,AU,AIB7,2021-01-29 22:11:00,-0.77,-4.50,3.101096,24.4,0 days 00:00:00
6512,TRAIN_006512,AU,AIB7,2021-01-29 22:11:00,-0.77,-4.50,3.101096,24.4,0 days 00:00:00
222572,TRAIN_222572,AU,AIB7,2021-03-01 02:36:00,-7.00,-8.50,5.577288,22.3,3 days 07:27:10.000008
331088,TRAIN_331088,AU,AIB7,2021-03-01 02:36:00,0.80,1.02,1.339677,23.0,0 days 00:00:00
29859,TRAIN_029859,AU,AIB7,2021-03-31 05:36:00,-4.02,-1.90,3.046966,23.9,5 days 03:00:29.000160
...,...,...,...,...,...,...,...,...,...
230163,TRAIN_230163,ZA,CXL1,2021-01-28 20:38:00,-1.03,8.88,4.853695,18.6,10 days 12:02:18.999960
50739,TRAIN_050739,ZA,CXL1,2021-04-29 11:06:00,-0.86,-2.60,2.205677,17.9,5 days 22:59:27.999960
343145,TRAIN_343145,ZA,CXL1,2021-04-29 11:06:00,1.15,-3.62,2.743175,16.1,5 days 14:00:01.000079999
208653,TRAIN_208653,ZA,CXL1,2023-02-06 11:33:00,0.42,9.50,5.057791,18.5,1 days 07:31:31.000008


In [8]:
dup_train['Bert_time'] = dup_train['ATA'] + dup_train['CI_HOUR']

In [9]:
dup_train[dup_train['ARI_CO'] == 'AU'].sort_values(by=['Bert_time']).head(20)

,SAMPLE_ID,ARI_CO,ARI_PO,ATA,U_WIND,V_WIND,BN,AIR_TEMPERATURE,CI_HOUR,Bert_time
51386,TRAIN_051386,AU,WHH4,2020-09-01 00:17:00,-0.23,2.44,2.048344,25.7,0 days 00:00:00,2020-09-01 00:17:00.000000
202905,TRAIN_202905,AU,WHH4,2020-09-01 00:17:00,-0.23,2.44,2.048344,25.7,0 days 00:00:00,2020-09-01 00:17:00.000000
57869,TRAIN_057869,AU,WHH4,2020-09-01 00:18:00,-0.23,2.44,2.048344,25.7,0 days 00:00:00,2020-09-01 00:18:00.000000
48228,TRAIN_048228,AU,WHH4,2020-09-01 00:18:00,1.37,1.58,1.842746,25.6,3 days 21:22:48,2020-09-04 21:40:48.000000
68359,TRAIN_068359,AU,WHH4,2020-10-03 15:11:00,1.01,3.34,2.592351,26.4,9 days 06:00:11.999880,2020-10-12 21:11:11.999880
362105,TRAIN_362105,AU,WHH4,2020-10-03 15:11:00,6.38,-0.87,3.900109,23.7,15 days 17:24:00,2020-10-19 08:35:00.000000
123553,TRAIN_123553,AU,NQO4,2020-10-21 21:41:00,-1.10,0.14,1.207213,20.6,0 days 00:00:00,2020-10-21 21:41:00.000000
315356,TRAIN_315356,AU,NQO4,2020-10-21 21:41:00,0.51,-3.83,2.774636,30.7,5 days 05:58:36.000120,2020-10-27 03:39:36.000120
129506,TRAIN_129506,AU,WHH4,2020-12-06 17:10:00,-4.74,5.11,4.111631,27.2,2 days 09:29:43.000008,2020-12-09 02:39:43.000008
171921,TRAIN_171921,AU,WHH4,2020-12-11 12:39:00,11.01,-1.20,5.598825,27.5,0 days 10:58:35.000004,2020-12-11 23:37:35.000004


## test

In [10]:
test = test[['SAMPLE_ID', 'ARI_CO', 'ARI_PO', 'ATA', 'U_WIND', 'V_WIND', 'BN', 'AIR_TEMPERATURE']]

In [11]:
test['ATA'] = pd.to_datetime( test['ATA'])

In [12]:
# train에서 ARI_CO, ARI_PO, ATA가 서로 같은 데이터를 찾아서 dup_train에 저장
dup_test = test[(test.duplicated(['ARI_CO', 'ARI_PO', 'ATA'], keep=False)) & (test['ATA'] > '2020-01-01')].sort_values(by=['ARI_CO', 'ARI_PO', 'ATA'])
dup_test

,SAMPLE_ID,ARI_CO,ARI_PO,ATA,U_WIND,V_WIND,BN,AIR_TEMPERATURE
32060,TEST_032060,AU,AIB7,2020-10-20 22:10:00,-4.92,-6.29,4.501968,20.1
48152,TEST_048152,AU,AIB7,2020-10-20 22:10:00,-3.39,-5.30,3.840273,18.2
105881,TEST_105881,AU,AIB7,2020-11-17 09:11:00,-7.20,0.19,4.202602,18.9
157695,TEST_157695,AU,AIB7,2020-11-17 09:11:00,-6.94,-4.38,4.584627,19.3
101157,TEST_101157,AU,AIB7,2022-02-10 12:59:00,-7.93,-4.80,4.972432,23.3
...,...,...,...,...,...,...,...,...
146136,TEST_146136,TW,JWI3,2022-11-22 21:10:00,-2.70,-4.41,3.369556,26.5
50131,TEST_050131,US,QGN3,2022-11-17 16:45:00,-0.38,-7.31,4.248133,11.9
102687,TEST_102687,US,QGN3,2022-11-17 16:45:00,-0.38,-7.31,4.248133,11.9
157778,TEST_157778,VN,JGL5,2020-09-15 00:32:00,-1.27,-0.42,1.368013,26.5


In [17]:
dup_test['year'] = dup_test['ATA'].dt.year
dup_test[dup_test['ARI_CO'] == 'AU'].sort_values(by=['ATA']).head(20)

,SAMPLE_ID,ARI_CO,ARI_PO,ATA,U_WIND,V_WIND,BN,AIR_TEMPERATURE,year
32060,TEST_032060,AU,AIB7,2020-10-20 22:10:00,-4.92,-6.29,4.501968,20.1,2020
48152,TEST_048152,AU,AIB7,2020-10-20 22:10:00,-3.39,-5.30,3.840273,18.2,2020
105881,TEST_105881,AU,AIB7,2020-11-17 09:11:00,-7.20,0.19,4.202602,18.9,2020
157695,TEST_157695,AU,AIB7,2020-11-17 09:11:00,-6.94,-4.38,4.584627,19.3,2020
94215,TEST_094215,AU,WHH4,2021-05-23 03:35:00,-11.53,2.56,5.844062,22.9,2021
34866,TEST_034866,AU,WHH4,2021-05-23 03:35:00,-11.56,1.49,5.792770,25.1,2021
75095,TEST_075095,AU,WHH4,2021-08-24 00:52:00,2.58,-3.79,3.109879,24.3,2021
61680,TEST_061680,AU,WHH4,2021-08-24 00:52:00,-0.12,3.07,2.381455,23.9,2021
101157,TEST_101157,AU,AIB7,2022-02-10 12:59:00,-7.93,-4.80,4.972432,23.3,2022
149739,TEST_149739,AU,AIB7,2022-02-10 12:59:00,-3.50,-2.62,3.012886,25.6,2022


## lab1

In [18]:
temp_train = dup_train[dup_train['ARI_CO'] == 'AU']
temp_train['year'] = temp_train['ATA'].dt.year
temp_train = temp_train.query('year == 2022')
#temp_train

C:\Users\pc\AppData\Local\Temp\ipykernel_35712\1236943172.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_train['year'] = temp_train['ATA'].dt.year


In [19]:
temp_train

,SAMPLE_ID,ARI_CO,ARI_PO,ATA,U_WIND,V_WIND,BN,AIR_TEMPERATURE,CI_HOUR,Bert_time,year
210328,TRAIN_210328,AU,AIB7,2022-02-10 02:59:00,-5.39,-7.79,5.044890,22.6,4 days 13:55:17.000040,2022-02-14 16:54:17.000040,2022
290959,TRAIN_290959,AU,AIB7,2022-02-10 02:59:00,1.17,-0.99,1.497925,24.0,0 days 00:00:00,2022-02-10 02:59:00.000000,2022
348574,TRAIN_348574,AU,AIB7,2022-02-14 02:55:00,0.93,-1.93,1.872666,21.8,22 days 00:03:21.999960,2022-03-08 02:58:21.999960,2022
373654,TRAIN_373654,AU,AIB7,2022-02-14 02:55:00,-6.96,10.83,6.189610,22.7,4 days 14:04:04.000080,2022-02-18 16:59:04.000080,2022
95735,TRAIN_095735,AU,AIB7,2022-02-15 02:59:00,-6.54,-5.35,4.674659,23.3,0 days 00:00:00,2022-02-15 02:59:00.000000,2022
...,...,...,...,...,...,...,...,...,...,...,...
178683,TRAIN_178683,AU,WXQ2,2022-05-16 18:58:00,-5.00,0.47,3.304578,25.1,2 days 11:54:33.000012,2022-05-19 06:52:33.000012,2022
268386,TRAIN_268386,AU,YDP4,2022-03-28 16:57:00,2.37,-1.46,2.229813,28.7,0 days 00:00:00,2022-03-28 16:57:00.000000,2022
342990,TRAIN_342990,AU,YDP4,2022-03-28 16:57:00,2.37,-1.46,2.229813,28.7,0 days 00:00:00,2022-03-28 16:57:00.000000,2022
115081,TRAIN_115081,AU,YDP4,2022-11-22 02:57:00,1.15,1.62,1.780797,25.5,2 days 20:48:12.999996,2022-11-24 23:45:12.999996,2022


In [20]:
temp_test = dup_test.query('year == 2022')
temp_test

,SAMPLE_ID,ARI_CO,ARI_PO,ATA,U_WIND,V_WIND,BN,AIR_TEMPERATURE,year
101157,TEST_101157,AU,AIB7,2022-02-10 12:59:00,-7.93,-4.80,4.972432,23.3,2022
149739,TEST_149739,AU,AIB7,2022-02-10 12:59:00,-3.50,-2.62,3.012886,25.6,2022
13845,TEST_013845,AU,AIB7,2022-07-18 02:28:00,10.04,3.38,5.435343,13.1,2022
32909,TEST_032909,AU,AIB7,2022-07-18 02:28:00,-6.86,2.76,4.276921,16.7,2022
4663,TEST_004663,AU,KSF1,2022-05-02 20:58:00,-0.31,2.93,2.315896,19.7,2022
...,...,...,...,...,...,...,...,...,...
51442,TEST_051442,TW,JWI3,2022-09-21 09:49:00,-6.47,-0.14,3.913210,28.3,2022
114899,TEST_114899,TW,JWI3,2022-11-22 21:10:00,-4.17,-1.80,3.090446,26.1,2022
146136,TEST_146136,TW,JWI3,2022-11-22 21:10:00,-2.70,-4.41,3.369556,26.5,2022
50131,TEST_050131,US,QGN3,2022-11-17 16:45:00,-0.38,-7.31,4.248133,11.9,2022


In [21]:
from datetime import datetime, timedelta

def excel_d2n(date_str):
    #print(date_str)
    date_obj = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    base_date = datetime(1899, 12, 30)
    delta = date_obj - base_date
    numeric_value = delta.total_seconds() / (60 * 60 * 24)  # 초를 일로 변환
    return numeric_value

def excel_n2d(numeric_value):
    # 1899-12-30에 일 수를 더한 날짜를 계산
    date_obj = datetime(1899, 12, 30) + timedelta(days=numeric_value)
    # 원하는 날짜 형식으로 문자열로 변환
    date_str = date_obj.strftime('%Y-%m-%d %H:%M:%S.%f')
    return date_str



In [22]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import LinearNDInterpolator,interp1d

x = temp_train['Bert_time'].apply(lambda x: excel_d2n(str(x).split('.')[0]))
#y = temp_train['V_WIND']
y = temp_train[['V_WIND', 'AIR_TEMPERATURE']]

# 그래프 그리기
#plt.plot(x, y, 'o', label='데이터 포인트')

# 보간 함수 만들기
#f = interp1d(y, x, kind='linear')
f = LinearNDInterpolator(y, x, fill_value=np.nan)

# 찾고자 하는 특정 y 값
target_y = temp_test.iloc[0,:][['V_WIND', 'AIR_TEMPERATURE']]  # 예시로 특정 y 값 지정

# 보간을 사용하여 특정 y 값에 대응하는 x 값 찾기
matching_x = f(target_y)

"""plt.plot(matching_x, target_y, 'ro', label=f'찾은 x 값: {matching_x}')
plt.legend()

plt.xlabel('x')
plt.ylabel('y')
plt.show()"""

print(f"찾고자 하는 y 값 {target_y}에 해당하는 x 값: {matching_x}")

찾고자 하는 y 값 V_WIND             -4.8
AIR_TEMPERATURE    23.3
Name: 101157, dtype: object에 해당하는 x 값: [44660.07143609]


In [23]:
numeric_value =  44660.07143609
date_string = excel_n2d(numeric_value)
print(f"숫자를 날짜로 변환: {date_string}")

숫자를 날짜로 변환: 2022-04-09 01:42:52.078176


In [26]:
temp_test.iloc[0,:]

SAMPLE_ID                  TEST_101157
ARI_CO                              AU
ARI_PO                            AIB7
ATA                2022-02-10 12:59:00
U_WIND                           -7.93
V_WIND                            -4.8
BN                            4.972432
AIR_TEMPERATURE                   23.3
year                              2022
Name: 101157, dtype: object

In [29]:
time_difference_seconds = (pd.to_datetime(date_string, format='%Y-%m-%d %H:%M:%S.%f') - temp_test.iloc[0,:]['ATA']).total_seconds()
time_difference_hours = time_difference_seconds/3600
time_difference_hours

1380.7311328266667